In [38]:
import urllib.request
import json
import pandas as pd
import sqlite3
from datetime import datetime

In [28]:
tba_key='SRT3N9afeExlLFcaOCSNu7chw5qLOnCfz0jE0FciGNAjyICAIvjTJrjoQoUCw8EX'
evkey = '2020wasno'

In [29]:
def getheader():
    return {'X-TBA-Auth-Key': tba_key, 'User-Agent': 'frc1318scouting'}

def get(endurl):
    auth_hdr = getheader()
    url = f'https://www.thebluealliance.com/api/v3/{endurl}'
    req = urllib.request.Request(url, headers = auth_hdr)
    
    with urllib.request.urlopen(req) as resp:
        resp_text = resp.read()
    
    return (resp_text)

In [30]:
def get_matches(event_key):
    return get(f'event/{str(event_key)}/matches/simple')

In [47]:
# matches_list = json.loads(get_matches(evkey))
# matches_list

In [46]:
# matches_dict = [{'match': match['key'], 'station': idx+1, 'team': team,'alliance': alliance, 'matchtime':
#                  match['predicted_time']} for match in matches_list 
#              for alliance in match['alliances']
#              for idx, team in enumerate(match['alliances'][alliance]['team_keys'])]
# matches_dict

In [33]:
# matchlist4 = [(match['key'],idx+1,team,alliance,match['predicted_time']) for match in matches_list 
#          for alliance in match['alliances']
#              for idx, team in enumerate(match['alliances'][alliance]['team_keys'])]
# matchlist4

In [15]:
def create_matches(dbname):
    con = sqlite3.connect(dbname)
    cur = con.cursor()
   
    matches_table = """ CREATE TABLE Matches (
                        match text,
                        station int,
                        team text,
                        alliance text,
                        matchtime text
    );
    """
    cur.execute(matches_table)
    con.commit()
    con.close()

def insert_into_matches(dbname):
    con = sqlite3.connect(dbname)
    cur = con.cursor()
    
    insert_matches = """ INSERT INTO Matches (match, station, team, alliance, matchtime)
                         values(?,?,?,?,?);
    """
    cur.executemany(insert_matches, matchlist4)
    con.commit()
    cur.close()
    
    


In [9]:
create_matches('testdb23')

In [16]:
insert_into_matches('testdb23')

In [36]:
def showdb(dbname):
    con=sqlite3.connect(dbname)
    cur=con.cursor()
    
    q1 = """ SELECT * FROM Matches
    """
    return(pd.read_sql_query(q1,con))

In [37]:
print(showdb('testdb23'))

               match  station     team alliance   matchtime
0     2020wasno_f1m1        1  frc2930     blue  1583104822
1     2020wasno_f1m1        2  frc2976     blue  1583104822
2     2020wasno_f1m1        3  frc4918     blue  1583104822
3     2020wasno_f1m1        1  frc4911      red  1583104822
4     2020wasno_f1m1        2  frc2910      red  1583104822
..               ...      ...      ...      ...         ...
529  2020wasno_sf2m2        2  frc2976     blue  1583103620
530  2020wasno_sf2m2        3  frc4918     blue  1583103620
531  2020wasno_sf2m2        1  frc2522      red  1583103620
532  2020wasno_sf2m2        2  frc3826      red  1583103620
533  2020wasno_sf2m2        3  frc8032      red  1583103620

[534 rows x 5 columns]
None


In [40]:
def unixtodt(utime):
    return datetime.utcfromtimestamp(utime).strftime('%Y-%m-%d %H:%M:%S')